# Import libs

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
%matplotlib inline 
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
sys.getdefaultencoding()
import re
import os
import commands

# Open comments

In [18]:
df = pd.read_csv("/Users/plotti/code/swissinfo-data/comments_with_id.csv")
df.head()

,text,creationDate,state.name,languageId,target.site,target.id,authorName,anonymousAddress,anonymoussEmail
0,swissinfoにご意見・ご感想をお送りいただくためには以下にあるフォームをご使用ください...,2009-11-17T08:22:49.896Z,APPROVED,ja,swissinfo-ja,coremedia:///cap/content/752124,ご感想をお送りいた,Cayman Islands,NaN
1,Suzanne! It is because of the fact that George...,2013-03-15T01:00:53.571Z,APPROVED,en,swissinfo-en,coremedia:///cap/content/45298,Rohil,United States,NaN
2,The Iraqi journalist is also in Switzerland to...,2009-11-18T09:03:14.710Z,APPROVED,en,swissinfo-en,coremedia:///cap/content/45298,Steve,United Kingdom,NaN
3,ad af afasf asf af as fasf asf asf asfasf,2009-11-25T15:10:00.329Z,REJECTED,en,swissinfo-en,coremedia:///cap/content/45298,Uwe & Leo,Switzerland,NaN
4,gfhnfg ofh fgnhu oiuhn gf,2009-12-01T11:08:33.468Z,REJECTED,fr,swissinfo-fr,coremedia:///cap/content/167184,Luc,Afghanistan,NaN


In [19]:
len(df)

91939

In [20]:
df_en = df[df["languageId"] == "en"]

In [21]:
len(df_en)

26857

# Make Categories

In [22]:
df_en["state.name"] = pd.Categorical(df_en["state.name"])
df_en['code'] = df_en["state.name"].cat.codes
df_en.head()

,text,creationDate,state.name,languageId,target.site,target.id,authorName,anonymousAddress,anonymoussEmail,code
1,Suzanne! It is because of the fact that George...,2013-03-15T01:00:53.571Z,APPROVED,en,swissinfo-en,coremedia:///cap/content/45298,Rohil,United States,NaN,0
2,The Iraqi journalist is also in Switzerland to...,2009-11-18T09:03:14.710Z,APPROVED,en,swissinfo-en,coremedia:///cap/content/45298,Steve,United Kingdom,NaN,0
3,ad af afasf asf af as fasf asf asf asfasf,2009-11-25T15:10:00.329Z,REJECTED,en,swissinfo-en,coremedia:///cap/content/45298,Uwe & Leo,Switzerland,NaN,1
5,"This is only a ban on minarets, not on mosques...",2009-12-14T18:14:04.780Z,APPROVED,en,swissinfo-en,coremedia:///cap/content/7793976,David,Canada,NaN,0
6,Nobody's exercise of individual human rights w...,2009-12-06T18:18:59.866Z,APPROVED,en,swissinfo-en,coremedia:///cap/content/7793976,Stefi,United States,NaN,0


In [24]:
dict( enumerate(df_en['state.name'].cat.categories) )

{0: 'APPROVED', 1: 'REJECTED'}

# Filter down only to approved and rejected

In [25]:
relevant_categories = ["APPROVED","REJECTED"]

In [26]:
df_filtered = df_en[df_en['state.name'].isin(relevant_categories)]
len(df_filtered)

26857

# How many are approved vs. rejected

In [28]:
df_filtered['state.name'].value_counts()

APPROVED    21097
REJECTED     5760
Name: state.name, dtype: int64

# Prepare data for training

In [29]:
import json
from pathlib import Path
import re
import random

training_data = Path("fasttext_comments_dataset_training.txt")
test_data = Path("fasttext_comments_dataset_test.txt")

# What percent of data to save separately as test data
percent_test_data = 0.10


def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

with training_data.open("w") as train_output, \
     test_data.open("w") as test_output:

    for i,item in df_filtered.iterrows():
        #print(item)
        if (item["state.name"] != "") & (item["text"] != ""):
            tag = item["code"]
            text = item["text"].replace("\n", " ")
            text = strip_formatting(text)

            fasttext_line = unicode("__label__{} {}".format(tag, text),'utf-8')

            if random.random() <= percent_test_data:
                test_output.write(fasttext_line + "\n")
            else:
                train_output.write(fasttext_line + "\n")

# Train

In [30]:
!fasttext supervised -input fasttext_comments_dataset_training.txt -output category_model 

# Test

In [31]:
!fasttext test category_model.bin fasttext_comments_dataset_test.txt

# Results

- We've got 80% hit rate without tuning. 
- Thats enough to screen comments I'd say.

# Predict Something

In [33]:
r = dict( enumerate(df_filtered['state.name'].cat.categories) )

In [36]:
out = commands.getstatusoutput("echo 'such a shit. I hate you all' | fasttext predict category_model.bin -")[1]
r[int(out.split("_")[-1])]

'REJECTED'

In [37]:
out = commands.getstatusoutput("echo 'A very nice comment with kisses and stuff.' | fasttext predict category_model.bin -")[1]
r[int(out.split("_")[-1])]

'APPROVED'

# Train with tuning 

https://www.analyticsvidhya.com/blog/2017/07/word-representations-text-classification-using-fasttext-nlp-facebook/

In [65]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 100 -lr 1.0

In [66]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.21 worse accuracy :(

In [27]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 1.0 -wordNgrams 3

In [28]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.32 better accuracy

In [29]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 1.0 -wordNgrams 2

In [30]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.35 

In [71]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 30 -lr 1.0 -wordNgrams 2

In [72]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.238

In [73]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 0.1 -wordNgrams 2

In [74]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.305

In [75]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 2.0 -wordNgrams 2

In [76]:
!fasttext test category_model.bin fasttext_dataset_test.txt

# Grid Search

In [ ]:
!./gridSearch.sh